<a href="https://colab.research.google.com/github/hihohub/umbrella-networks/blob/main/sigmoid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Umbrella networks
- copyright 2021 by James Smith
- contact - derekjsmit@gmail.com

Demonstration instructions
- How to organize a sigmoid umbrella network, which is an ensemble of binary sigmoid predictors.
- Organize your image files into folders by category. For an ensemble of single-taskers, traditionally, only one level of folders is used. However, multiple levels are also possible, where an ensemble of single-taskers is formed for each level. For example, (book covers) -> (scifi), (fantasy), (mystery) would traditionally be (scifi book covers), (fantasy book covers), (mystery book covers).
- First, run the large cell containing the umbrella code, then scroll down 1333 lines to the experiment.
- Change the folder name in the demonstration code to your folder path.
- The rest of the code should work as written.
- Set 'levels' to anywhere from 1 to the number of layers in your folder structure. If levels=1, images from lower level folders are compressed into the outer level folders. If levels > 1, umbrella networks are formed recursively, one for each outer folder and each subfolder within the outer folders, up to the specified number of levels. The inner folders do not each require the same number of levels, but must have at least as many levels as the value specified for 'levels'.

Umbrella network instructions<br/>
- The umbrella code was designed for reading a local folder and automatically constructing the hierarchical network from the implicit hierarchy in the subfolders.
- If have images in folders, initialize with images_folder = path. Umbrella network constructs tree by itself. Refer to examples.
- However, it also has functions for reading the data from an online repository and building the tree node by node (since the data has no folder structure).
- If loading data from repository, initialize with images_folder = None, then use repo functions. Must first preprocess the data (normalize, reshape). Only process x data, not labels. Umbrella network constructs labels by itself. However, subdivide classes by label, then manually construct tree with repo functions. Finally, init_from_repo(). Refer to examples.
- In repo mode, umbrella network throws away labels and makes its own. For example, if divide MNIST into odd and even numbers, each subset will have labels 0-4. Hence, anything other than flat topology requires that the umbrella network form new labels.






In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


%%%%%%%%%%%%%%%%%%<br/>
UMBRELLA NETWORK<br/>
%%%%%%%%%%%%%%%%%%<br/>

In [2]:
! wget https://raw.githubusercontent.com/hihohub/umbrella-networks/main/umbrella.py

--2022-04-15 04:46:30--  https://raw.githubusercontent.com/hihohub/umbrella-networks/main/umbrella.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 48219 (47K) [text/plain]
Saving to: ‘umbrella.py’

umbrella.py         100%[===================>]  47.09K  --.-KB/s    in 0.01s   

2022-04-15 04:46:30 (4.72 MB/s) - ‘umbrella.py’ saved [48219/48219]



In [3]:
import umbrella
from umbrella import Umbrella_Network,Umbrella_Label,Umbrella_Node,Joint_Probability_Network

In [4]:
import os
import sys
import numpy as np
import math
import random
from PIL import Image, ImageOps, ImageChops
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Flatten, Dense, add, BatchNormalization, Dropout, AveragePooling2D, GlobalAveragePooling2D, SeparableConv2D
from tensorflow.keras.utils import to_categorical
from tensorflow.python.framework.ops import disable_eager_execution
from tensorflow.keras import backend as backend
from keras.callbacks import ReduceLROnPlateau
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import pickle
import copy
import time

SIGMOID ON SMALL MULTI-LAYER STRUCTURE COMPRESSED INTO TOP LEVEL FOLDERS
- the root folder contains 3 inner folders, each of which contains several inner folders
- layers is set to just 1, so the program makes just 3 sigmoid models, compressing their inner folders into 1 logical folder
- each model trains on its own class with an equal number of random samples from the other classes
- each model outputs 1 sigmoid probability for its own class

In [5]:
# the small_hierarchy folder has a just 3 outer folders, each with some inner folders
network_type = 'sigmoid'
images_folder = "/content/drive/My Drive/5390/project/small_hierarchy"
serial_file = "/content/drive/My Drive/Colab Notebooks/small_hierarchy.dat"
models_folder = "/content/drive/My Drive/Colab Notebooks/models"
target_image_size = 150
levels = 1 
channels = 3
valid_ratio = 0.2 # validation set
test_ratio = 0.0 # test set
initialize_now = True
SIGMOID = Umbrella_Network(network_type,images_folder,serial_file,models_folder,target_image_size,channels,levels,valid_ratio,test_ratio,initialize_now)


In [6]:
SIGMOID.traverse_training_data()

small_hierarchy trainX 0 (0,) trainY 0 (0,)
   animals trainX 71 (71,) trainY 71 (71,)
      alligator trainX 0 (0,) trainY 0 (0,)
      mammals trainX 0 (0,) trainY 0 (0,)
      salamanders trainX 0 (0,) trainY 0 (0,)
      snake trainX 0 (0,) trainY 0 (0,)
   objects trainX 103 (103,) trainY 103 (103,)
      automobiles trainX 0 (0,) trainY 0 (0,)
      bicycles trainX 0 (0,) trainY 0 (0,)
      boats trainX 0 (0,) trainY 0 (0,)
   plants trainX 103 (103,) trainY 103 (103,)
      cactus trainX 0 (0,) trainY 0 (0,)
      hedges trainX 0 (0,) trainY 0 (0,)
      shrubs trainX 0 (0,) trainY 0 (0,)


In [7]:
SIGMOID.make_models()

making model for animals
making model for objects
making model for plants


In [8]:
SIGMOID.train_models(eps=1,batch=10,aug=False,vrb=True)

training model for animals
trainX 71
trainY 71
8/8 [==============================] - 12s 76ms/step - loss: 1.2612 - accuracy: 0.4366
training model for objects
trainX 103
trainY 103
11/11 [==============================] - 1s 31ms/step - loss: 1.1526 - accuracy: 0.5146
training model for plants
trainX 103
trainY 103
11/11 [==============================] - 1s 21ms/step - loss: 0.7158 - accuracy: 0.6117


In [9]:
SIGMOID.get_training_accuracy()
SIGMOID.predict_validation_set() # set verbose=True to view all the images
SIGMOID.predict_validation_top5()

average training accuracy 0.520944
average training loss 1.043200
34

average validation accuracy 0.000000
top 5 accuracy 0.18


SIGMOID ON SMALL MULTI-LAYER STRUCTURE WITHOUT COMPRESSION OF LOWER LAYERS
- the root folder contains 3 inner folders, each of which contains several inner folders, which makes a total of 2 layers
- layers is set to 2, so the program makes 3 outer models and 3-4 inner models for each outer model
- each model trains on its own class with an equal number of random samples from the other classes on the same level
- each model outputs 1 sigmoid probability for its own class

In [10]:
# the small_hierarchy folder has a just 3 outer folders, each with some inner folders
network_type = 'sigmoid'
images_folder = "/content/drive/My Drive/5390/project/small_hierarchy"
serial_file = "/content/drive/My Drive/Colab Notebooks/small_hierarchy.dat"
models_folder = "/content/drive/My Drive/Colab Notebooks/models"
target_image_size = 150
channels = 3
levels = 2
valid_ratio = 0.2 # validation set
test_ratio = 0.0 # test set
initialize_now = True
SIGMOID = Umbrella_Network(network_type,images_folder,serial_file,models_folder,target_image_size,channels,levels,valid_ratio,test_ratio,initialize_now)


In [11]:
SIGMOID.traverse_training_data()

small_hierarchy trainX 0 (0,) trainY 0 (0,)
   animals trainX 71 (71,) trainY 71 (71,)
      alligator trainX 11 (11,) trainY 11 (11,)
      mammals trainX 29 (29,) trainY 29 (29,)
      salamanders trainX 17 (17,) trainY 17 (17,)
      snake trainX 17 (17,) trainY 17 (17,)
   objects trainX 103 (103,) trainY 103 (103,)
      automobiles trainX 35 (35,) trainY 35 (35,)
      bicycles trainX 28 (28,) trainY 28 (28,)
      boats trainX 40 (40,) trainY 40 (40,)
   plants trainX 103 (103,) trainY 103 (103,)
      cactus trainX 23 (23,) trainY 23 (23,)
      hedges trainX 35 (35,) trainY 35 (35,)
      shrubs trainX 46 (46,) trainY 46 (46,)


In [12]:
SIGMOID.make_models()

making model for animals
making model for alligator
making model for mammals
making model for salamanders
making model for snake
making model for objects
making model for automobiles
making model for bicycles
making model for boats
making model for plants
making model for cactus
making model for hedges
making model for shrubs


In [13]:
SIGMOID.train_models(eps=1,batch=10,aug=False,vrb=True)

training model for animals
trainX 71
trainY 71
8/8 [==============================] - 1s 20ms/step - loss: 0.8653 - accuracy: 0.5352
training model for alligator
trainX 11
trainY 11
2/2 [==============================] - 1s 11ms/step - loss: 0.7974 - accuracy: 0.3636
training model for mammals
trainX 29
trainY 29
3/3 [==============================] - 1s 125ms/step - loss: 1.4296 - accuracy: 0.4828
training model for salamanders
trainX 17
trainY 17
2/2 [==============================] - 1s 194ms/step - loss: 0.7348 - accuracy: 0.4706
training model for snake
trainX 17
trainY 17
2/2 [==============================] - 1s 18ms/step - loss: 1.6310 - accuracy: 0.3529
training model for objects
trainX 103
trainY 103
11/11 [==============================] - 1s 20ms/step - loss: 1.4817 - accuracy: 0.5340
training model for automobiles
trainX 35
trainY 35
4/4 [==============================] - 1s 61ms/step - loss: 2.0030 - accuracy: 0.4857
training model for bicycles
trainX 28
trainY 28
3/3 [==

In [14]:
SIGMOID.get_training_accuracy()
SIGMOID.predict_validation_set() # set verbose=True to view all the images
SIGMOID.predict_validation_top5()

average training accuracy 0.442309
average training loss 1.409272
34

average validation accuracy 0.147059
top 5 accuracy 0.68
